In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA


In [3]:

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X_rows, y = mnist.data, mnist.target.astype(int)

# Transpose X so each column is a data point (784 x 70000)
X = X_rows.T 

digits = list(range(10))
digit_matrices = {}

# Creating list of means for all digits
digit_means = []
for j in digits:
    # Select columns corresponding to digit j
    digit_matrices[f'X_{j}'] = X[:, y == j]
    # Calculate mean across columns (axis=1)
    digit_means.append(np.mean(digit_matrices[f'X_{j}'], axis=1))

digit_means = np.array(digit_means).T  
# Add overall mean as another column
overall_mean = np.mean(X, axis=1).reshape(-1, 1) 


mu_mat = np.hstack([digit_means, overall_mean]) 

/Users/cristianzuniga/Library/Python/3.10/lib/python/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
#CHecking shape to make sure its correct:
mu_mat.shape

(784, 11)

In [8]:
average_differences = []
for i in range(10):
    avg_dif = np.mean(mu_mat[i] - mu_mat[-1])
    average_differences.append(avg_dif)
    print(f"Difference between mean digit {i} and MNIST mean: {avg_dif}")
    
    
print()
print(f"Average mean difference between mu_j and mu: {np.mean(average_differences)}")
print(f"Standard deviation of mean differences between mu_j and mu: {np.std(average_differences)}")

Difference between mean digit 0 and MNIST mean: 0.0
Difference between mean digit 1 and MNIST mean: 0.0
Difference between mean digit 2 and MNIST mean: 0.0
Difference between mean digit 3 and MNIST mean: 0.0
Difference between mean digit 4 and MNIST mean: 0.0
Difference between mean digit 5 and MNIST mean: 0.0
Difference between mean digit 6 and MNIST mean: 0.0
Difference between mean digit 7 and MNIST mean: 0.0
Difference between mean digit 8 and MNIST mean: 0.0
Difference between mean digit 9 and MNIST mean: 0.0

Average mean difference between mu_j and mu: 0.0
Standard deviation of mean differences between mu_j and mu: 0.0


In [10]:
#Performing PCA on the union of the means
mu_mat_centered = mu_mat - np.mean(X)
pca_mu = PCA(n_components=10).fit(mu_mat_centered)


#PCA on X
X_centered = X - np.mean(X)
pca_X = PCA(n_components=10).fit(X_centered)


In [11]:
def random_vectors(V, samples):
    coeffs = np.random.rand(V.shape[1], samples)
    coeffs /= np.linalg.norm(coeffs, axis=0)
    
    return V @ coeffs

def orthogonal_projection_length(V, V_P, samples):
    projections = []
    i = 0
    while i < samples:
        projections.append(np.linalg.norm(V @ random_vectors(V_P, 1)))
        i += 1
    return projections

In [12]:

#Getting the principal component matrices
subspace_pca_mu = pca_mu.components_
subspace_pca_X = pca_X.components_

#Building the projectiion matrix i.e QQ^T
proj_mat = subspace_pca_X @ subspace_pca_X.T

#Getting 1000 random vectors
projection_lengths = orthogonal_projection_length(proj_mat, subspace_pca_mu, 1000)

print(f"Mean projection length: {np.mean(projection_lengths):.4f} ± {np.std(projection_lengths):.4f}")

Mean projection length: 0.9876 ± 0.0141


In [ ]:
#Solving the Orthogonal Prosecutes Problem

R = subspace_pca_mu.T @ subspace_pca_X

U, S, V_T = np.linalg.svd(R, full_matrices=False)

Q = U @ V_T

distance = np.linalg.norm((subspace_pca_X - subspace_pca_mu @ Q), ord='fro')
print(f"The distance between the two subspaces is: {distance:.10f}")

The Distance between the two subspaces is 0.0000000000
